In [ ]:
import tifffile
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from tqdm import tqdm
import os
from skimage.measure import regionprops
import sys

# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add the parent directory to sys.path
sys.path.append(parent_dir)

# aux

In [ ]:
def read_sequence(folder_path):
    """
    Read a sequence of TIFF files in a folder as a 3D volume.

    Args:
    folder_path (str): Path to the folder containing TIFF files.

    Returns:
    numpy.ndarray: A 3D array where each slice corresponds to a TIFF file.
    """

    # List and sort the TIFF files
    tiff_files = sorted(
        [
            os.path.join(folder_path, f)
            for f in os.listdir(folder_path)
            if (f.endswith(".tiff") or f.endswith(".tif"))
        ]
    )

    tiff_sequence = tifffile.TiffSequence(tiff_files)

    # Get the total number of TIFF files
    total_files = len(tiff_files)

    # Read each TIFF file and update progress
    volume = []
    with tqdm(total=total_files, desc="Progress") as pbar:
        for i, file_path in enumerate(tiff_files):
            slice_data = tifffile.imread(file_path)
            volume.append(slice_data)

            # Update progress
            pbar.update(1)

    return np.array(volume)

# Main

In [ ]:
volume_path = Path(
    r"\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\medidas\JI_4+5+7+8\volumen_eq"
)

volume = read_sequence(volume_path)

print(volume.shape)

In [ ]:
plt.imshow(volume[0], cmap="gray")

In [ ]:
n_samples = 4

In [ ]:
# otsu threshold
from skimage.filters import threshold_otsu

thresh = threshold_otsu(volume)
binary = volume > thresh

plt.imshow(binary[0], cmap="gray")

In [ ]:
# binary fill holes
from scipy.ndimage import binary_fill_holes
from joblib import Parallel, delayed

# Assuming binary is already defined
filled = np.zeros_like(binary)


def process_slice(i):
    return binary_fill_holes(binary[i])


# Parallel processing
filled_slices = Parallel(n_jobs=-1)(
    delayed(process_slice)(i) for i in tqdm(range(binary.shape[0]))
)

# Combine the results
for i in range(binary.shape[0]):
    filled[i] = filled_slices[i]

plt.imshow(filled[0], cmap="gray")

In [ ]:
plt.imshow(filled.min(axis=0), cmap="gray")

In [ ]:
# binary dilate
from skimage.measure import label

label_image = label(filled)

props = regionprops(label_image)

# sort props by area
props.sort(key=lambda x: x.area, reverse=True)

In [28]:
def process_sample(volume, label_image, label, bbox):
    sample = volume.copy()
    sample[label_image != label] = 0
    sample = sample[bbox[0] : bbox[3], bbox[1] : bbox[4], bbox[2] : bbox[5]]
    return sample


volumes = Parallel(n_jobs=-1)(
    delayed(process_sample)(volume, label_image, props[i].label, props[i].bbox)
    for i in tqdm(range(n_samples))
)

100%|██████████| 4/4 [00:00<00:00, 795.96it/s]


In [ ]:
bboxes = [props[i].bbox for i in range(n_samples)]

# order the samples by bbox[2]

volumes = [v for _, v in sorted(zip(bboxes, volumes), key=lambda pair: pair[0][2])]